In [1]:
import pandas as pd # data frames
import numpy as np

In [2]:
# read data
df = pd.read_csv("https://raw.githubusercontent.com/askoshiyama/mli-cohort3/master/boston.csv")
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,T1
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


Repeated K Fold CV (5 outer, 5 inner, repeat 5 times):
RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.
https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators

In [3]:
# defining vars
input_vars = ["V1", "V2", "V3", "V4", "V5", "V6", "V7",
              "V8", "V9", "V10", "V11", "V12", "V13"]
target = ["T1"]

In [46]:
X = df.drop(labels=target, axis=1)
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33


In [45]:
y = df.drop(labels=input_vars, axis=1)
y.head()

,T1
0,24.0
1,21.6
2,34.7
3,33.4
4,36.2


In [78]:
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from interpret.glassbox import ExplainableBoostingRegressor 
from xgboost import XGBRegressor
from sklearn.svm import SVR

# define the models for selection along with their hyperparameters
# we try to find meaningful ranges for each hyperparameter. Among the models chosen for selection,
# we also include an MLP neural network as it presents the most recent advancement in supervised learning domain.
models_and_parameters = {
    'dummy': (DummyRegressor(), {}),
    'linReg': (LinearRegression(), {}),
    'ridge': (Ridge(), 
              {'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'svr': (SVR(kernel='rbf'),
              {'C': [0.01, 0.05, 0.1, 0.5, 1], 
               'gamma': [0.001, 0.0001, 0.00001]}),
    'krr': (KernelRidge(), 
              {'kernel': ["poly","rbf"],
               'degree': [2,3,4],
               'alpha': np.linspace(0.00001, 0.8, num=50)}),
    'dtr': (DecisionTreeRegressor(),
              {"min_samples_split": [2, 10, 20, 40], 
               "max_depth": [2, 6, 8],
               "min_samples_leaf": [1, 20, 40, 100],
               "max_leaf_nodes": [None, 5, 20, 100]}),
    'rf': (RandomForestRegressor(n_estimators=100),
              {'max_depth': [5, 10, 50, 100, 200, 500]}),
    'gbr': (GradientBoostingRegressor(),
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
    'ebr': (ExplainableBoostingRegressor(), {}),
    'xgbr': (XGBRegressor(), 
              {"max_depth": [2, 6, 8],
               "learning_rate": [0.1, 0.01, 0.001],
               "n_estimators": [50, 100, 150]}),
}
#     'mlp': (MLPRegressor(early_stopping=True),
#                {'mlp__hidden_layer_sizes': [(2,),(4,),(8,),(16,),(32,),(64,),(128,)],
#                 'mlp__activation': ['relu'],
#                 'mlp__solver':['lbfgs', 'adam', 'sgd'], 
#                 'mlp__alpha':[0.0001],
#                 'mlp__batch_size':['auto'], 
#                 'mlp__learning_rate':['constant'],
#                 'mlp__learning_rate_init':[0.01, 0.001, 0.0001], 
#                 'mlp__max_iter':[800]})

In [79]:
from sklearn import metrics

# performance metrics
perf_metrics = {"MAE": metrics.mean_absolute_error, 
                "EVS": metrics.explained_variance_score, 
                "MSE": metrics.mean_squared_error,
                "MDAE": metrics.median_absolute_error, 
                "R2": metrics.r2_score
               }
metric_greater_is_better = \
               {"MAE": False, 
                "EVS": True, 
                "MSE": False,
                "MDAE": False, 
                "R2": True
               }
scorer_metrics = {}
for pf in perf_metrics:
    scorer_metrics[pf] = metrics.make_scorer(score_func=perf_metrics[pf], greater_is_better=metric_greater_is_better[pf])

In [24]:
# based on: https://datascience.stackexchange.com/questions/13185/nested-cross-validation-and-selecting-the-best-regression-model-is-this-the-ri
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics



#######################################
# configuration
REPEAT = 3
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
METRIC = 'neg_mean_squared_error'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_cv = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_cv = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

avg_scores = dict()
for name in models_and_parameters.keys():
    avg_scores[name] = dict()
    avg_scores[name]['scores'] = []


for i in range(1, REPEAT+1):
    for name, (model, params) in models_and_parameters.items():    
        print(f"{i} - {name}")
        # first scale the data, e.g. required for SVM, Ridge
        steps = [('scale', StandardScaler()), (name, model)]
        pipe = Pipeline(steps)
        
        # choose best hyperparameters for model using KFold inner_cv
        model_with_gridsearch = GridSearchCV(
            estimator=pipe, 
            param_grid=params, 
            iid=False,
            cv=inner_cv, 
            scoring=METRIC
        )        

        # estimate generalization error on the K-fold splits of the data
        # average test set scores over several dataset splits (defined by outer_cv)
        scores_across_outer_folds = cross_val_score(
            model_with_gridsearch,
            X, y.values.ravel(), 
            cv=outer_cv, 
            scoring=METRIC)
        
        avg_score_model = np.mean(scores_across_outer_folds)
        avg_scores[name]['scores'].append(avg_score_model)
        # print(f"[{i}] Model: {name}:\tAverage MSE in the outer folds: {avg_score_model}")
        # print()

print()
print("Avg scores per model over 5 repetitions:")
for name, vals in avg_scores.items():
    scores = vals['scores']
    avg_score = np.mean(scores)
    print(f"{name}:\t{avg_score}") # \tscores: {scores}
    avg_scores[name]['avg_score'] = avg_score
    
many_stars = '\n' + '*' * 20 + '\n'
print(many_stars + 'choose the best model and refit on the whole dataset' + many_stars)

print(many_stars + '\n' + str(avg_scores) + '\n' + many_stars)

best_model_name, best_model_avg_score = max(
    avg_scores.items(),
    key=(lambda name_s_as: name_s_as[1]['avg_score']))

print(f"Best model: {best_model_name}\nBest avg score: {best_model_avg_score['avg_score']}")
      
best_model, best_model_params = models_and_parameters[best_model_name]
      
# now we refit this best model on the whole dataset so that we can start
# making predictions on other data, and now we have a reliable estimate of
# this model's generalization error and we are confident this is the best model
# among the ones we have tried
steps = [('scale', StandardScaler()), (best_model_name, best_model)]
pipe = Pipeline(steps)
final_regressor = GridSearchCV(
    estimator=pipe,
    param_grid=best_model_params,
    iid=False,
    cv=inner_cv, 
    scoring=METRIC)
  
final_regressor.fit(X, y.values.ravel())

print('Best model: \n\t{}'.format(best_model), end='\n\n')
print('Estimation of its generalization error (negative mean squared error):\n\t{}'.format(
    best_model_avg_score), end='\n\n')
print('Best parameter choice for this model: \n\t{params}'
      '\n(according to cross-validation `{cv}` on the whole dataset).'.format(
      params=final_regressor.best_params_, cv=inner_cv))



1 - linReg
1 - ridge
1 - svr
1 - krr
1 - dtr
1 - rf
1 - gbr
1 - ebr

Avg scores per model over 5 repetitions:
linReg:	-24.74043147037029
ridge:	-23.356545040361322
svr:	-64.20733923678839
krr:	-12.977789298691993
dtr:	-24.103768310299422
rf:	-11.828537843586556
gbr:	-10.309008689793977
ebr:	-13.778974373615373

********************
choose the best model and refit on the whole dataset
********************


********************

{'linReg': {'scores': [-24.74043147037029], 'avg_score': -24.74043147037029}, 'ridge': {'scores': [-23.356545040361322], 'avg_score': -23.356545040361322}, 'svr': {'scores': [-64.20733923678839], 'avg_score': -64.20733923678839}, 'krr': {'scores': [-12.977789298691993], 'avg_score': -12.977789298691993}, 'dtr': {'scores': [-24.103768310299422], 'avg_score': -24.103768310299422}, 'rf': {'scores': [-11.828537843586556], 'avg_score': -11.828537843586556}, 'gbr': {'scores': [-10.309008689793977], 'avg_score': -10.309008689793977}, 'ebr': {'scores': [-13.778974373615

ValueError: For multi-metric scoring, the parameter refit must be set to a scorer key or a callable to refit an estimator with the best parameter setting on the whole data and make the best_* attributes available for that metric. If this is not needed, refit should be set to False explicitly. True was passed.

In [71]:
# based on: https://datascience.stackexchange.com/questions/13185/nested-cross-validation-and-selecting-the-best-regression-model-is-this-the-ri
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold, KFold, cross_val_score, GridSearchCV
import numpy as np
from sklearn import metrics
from time import time


#######################################
# configuration
REPEAT = 5
SEED = None  # will ensure different results on each iteration
K_FOLD_NUM = 5
REFIT_METRIC = 'MSE'
#######################################

# use nested cross-validation for grid search and validation
# creates 5 folds for estimating generalization error
outer_folds = RepeatedKFold(n_repeats=REPEAT, n_splits=K_FOLD_NUM, random_state=SEED)

# when we train on a certain fold, we use a second cross-validation
# split in order to choose hyperparameters
inner_folds = KFold(n_splits=K_FOLD_NUM, random_state=SEED, shuffle=True)

i, z, innercv_results, inner_best_model = 0, 0, {}, {}
df_outer_results = pd.DataFrame(index=[0], columns=["Model", "Time"] + list(perf_metrics.keys()))

X_arr, y_arr = X.values, y.values

# outer loop
for (out_train, out_test) in outer_folds.split(X_arr, y_arr):
    print(f"Repetition {int(i / K_FOLD_NUM) + 1} Fold {i % K_FOLD_NUM + 1}")
    i += 1  # iteration
    # separation: train, test
    Xtrain, Xtest = X_arr[out_train], X_arr[out_test]
    ytrain, y_test = y_arr[out_train], y_arr[out_test]
    
    # inner loop -- all models
    fold_id = "fold_" + str(i)
    innercv_results[fold_id] = {}
    inner_best_model[fold_id] = {}
    start, end = [], []
    
    for model_name, (model, params) in models_and_parameters.items():
        start.append(time())
        print("\t", model_name)
        innercv_results[fold_id][model_name] = GridSearchCV(estimator=model, 
                                                            param_grid=params, 
                                                            scoring=scorer_metrics, 
                                                            iid=False,
                                                            cv=inner_folds, 
                                                            refit=REFIT_METRIC)    
        innercv_results[fold_id][model_name].fit(Xtrain, ytrain.ravel())
        inner_best_model[fold_id][model_name] = innercv_results[fold_id][model_name].best_estimator_
        end.append(time())        
    
    # outer loop prediction    
    for v, (model_name, (model, _)) in enumerate(models_and_parameters.items()):
        # prediction
        y_test_pred = inner_best_model[fold_id][model_name].predict(Xtest)
        df_outer_results.loc[z, "Model"] = model_name
        df_outer_results.loc[z, "Time"] = end[v] - start[v]
        
        # compute performance metrics
        for pf in perf_metrics.keys():
            df_outer_results.loc[z, pf] = perf_metrics[pf](y_test, y_test_pred)
        z += 1        

# final organisation
df_outer_results[list(perf_metrics.keys()) + ["Time"]] = df_outer_results[list(perf_metrics.keys()) + ["Time"]].astype(float)
print("done.")

Repetition 1 Fold 1
dummy
linReg
ridge
svr
krr


C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:189: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "
C:\dev\python\Anacon

...

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=5.59722e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.56308e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.30348e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.77465e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.37415e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=6.31354e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.29005e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.19412e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.06717e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=7.56525e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=5.59722e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.56308e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=4.30348e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.77465e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.09981e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=8.37769e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.30322e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.39953e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=1.18821e-17): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\envs\mli3\lib\site-packages\sklearn\linear_model\ridge.py:187: LinAlgWarning: Ill-conditioned matrix (rcond=9.54711e-18): result may not be accurate.
  overwrite_a=False)
C:\dev\python\Anaconda3\e

dtr
rf
gbr
ebr
done.


In [76]:
df_outer_results.pivot_table(index=["Model"], aggfunc="mean")

,EVS,MAE,MDAE,MSE,R2,Time
Model,,,,,,
dtr,7.497370e-01,2.895115,2.000236,21.149537,0.744540,2.706508
dummy,-4.440892e-17,6.654536,4.877815,84.609455,-0.006391,0.010771
ebr,8.487587e-01,2.432477,1.751214,12.791241,0.847647,38.654355
gbr,8.722324e-01,2.290820,1.758186,10.631168,0.870697,13.207089
krr,8.283218e-01,2.572445,1.861859,14.200137,0.826749,36.348141
linReg,7.152051e-01,3.445210,2.591385,23.967706,0.712309,0.017952
rf,8.654112e-01,2.282095,1.616082,11.310744,0.862563,8.737135
ridge,7.148679e-01,3.444873,2.591737,24.002299,0.711901,0.665433
svr,3.215883e-01,4.917003,3.024066,60.964573,0.276597,0.874925
